In [1]:
%%time

import pandas as pd
from Nummobility.core.TrajectoryDF import NumPandasTraj
from Nummobility.semantics.semantic_features import SemanticFeatures

from Nummobility.features.spatial_features import SpatialFeatures
import Nummobility.utilities.constants as const
from Nummobility.utilities.DistanceCalculator import FormulaLog

"""
    First of all, lets import all the datasets one by one
    and check out a few of their points.
"""
# Reading the geolife dataset and converting to NumPandasTraj.
# Also, lets, print the first 5 points of the dataset to
# see how the dataframe looks.
geolife = pd.read_csv('./data/geolife_sample.csv')
geolife = NumPandasTraj(geolife,'lat','lon','datetime','id')
geolife.head()

CPU times: user 2.55 s, sys: 208 ms, total: 2.76 s
Wall time: 2.77 s


lat         lon
traj_id DateTime                                  
1       2008-10-23 05:53:11  39.984224  116.319402
        2008-10-23 05:53:16  39.984211  116.319389
        2008-10-23 05:53:21  39.984217  116.319422
        2008-10-23 05:53:23  39.984710  116.319865
        2008-10-23 05:53:28  39.984674  116.319810

In [2]:
small = geolife.reset_index().loc[geolife.reset_index()['traj_id'] == '1']
small = NumPandasTraj(small,
                      latitude='lat',
                      longitude='lon',
                      datetime='datetime',
                      traj_id='traj_id')
small.head()

lat         lon
traj_id DateTime                                  
1       2008-10-23 05:53:11  39.984224  116.319402
        2008-10-23 05:53:16  39.984211  116.319389
        2008-10-23 05:53:21  39.984217  116.319422
        2008-10-23 05:53:23  39.984710  116.319865
        2008-10-23 05:53:28  39.984674  116.319810

In [3]:
SpatialFeatures.get_distance_travelled_by_traj_id(small, '1')

1024749.3231767347

In [4]:
%%time

sem = SemanticFeatures.nearest_bank_alt(small)
print(len(sem))

502
CPU times: user 5.8 s, sys: 0 ns, total: 5.8 s
Wall time: 5.8 s


In [5]:
start = small[const.LAT][0], small[const.LONG][0]
a = []
for i in range(len(sem)):
    dist = FormulaLog.haversine_distance(start[0], start[1],
                                         sem[i][0], sem[i][1])
    a.append(dist)

(pd.Series(a) > 1000).value_counts()


True    502
dtype: int64